In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Nov_22_10:17:15_PST_2023
Cuda compilation tools, release 12.3, V12.3.107
Build cuda_12.3.r12.3/compiler.33567101_0


In [9]:
!pip install llama-cpp-python \
  --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu123

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu123


In [ ]:
from llama_cpp import Llama
import tqdm

llm = Llama.from_pretrained(
    repo_id="bartowski/Meta-Llama-3.1-8B-Instruct-GGUF",
    filename="Meta-Llama-3.1-8B-Instruct-Q8_0.gguf",
    n_ctx = 4096
)


questions = [
    """Here's the idea: "{}"
What genre(s) could this story fall into? Answer in 1 sentence.
""",
    "What kind of message do you want the story to have? Answer in 1 sentence.",
    "Are there any sub-themes you want to incorporate? Answer in 1 sentence.",
    "Summarize the plot of the story for me in a single line."
    "In what time period does your story take place? Answer in 1 sentence.",
    "What's the primary setting or location? Answer in 1 sentence.",
    "Are there any unique aspects of this world that affect the story? Answer in 1 sentence.",
    "Are there any social, political, or technological elements that are important? Answer in 1 sentence.",
    "Who is your protagonist? What do they want?",
    "What obstacles stand in their way?",
    "Who are the other key characters?",
    "What are the relationships between characters?",
    "What motivates each character's actions?",
    "What flaws or quirks make your characters interesting?",
    "What inciting incident sets the story in motion? Answer in 1 sentence.",
    "What are the key events or turning points?",
    "What is the main conflict (internal, external, or both)?",
    "How will the tension escalate?",
    "What's at stake for the characters?",
    "How might the story end? What changes for the characters?",
    "Which scene or moment will you focus on?",
    "What context can you imply rather than state directly?",
    "Which details are essential to include?",
    "What sensory elements can quickly establish the setting?",
    """Make me an outline for making a flash fiction story using all of this information. 
This story will have a maximum word limit of 700-1000 words; so you would only be able to focus on the one scene. Having said that, break this scene up into beats for the outline. 
Do not plan for sudden jumps in time/location between these beats, unless absolutely necessary.

Add some details about the characters, and the setting of this scene at the end. Use a maximum of 500 words.

""",
    "Write a story of between 700-1000 words using this outline."
]



def make_story(idea):
    qs = list(questions)
    qs[0] = qs[0].format(idea)
    
    messages = [
        {'role': 'system', 'content': 'Over the next couple of messages, I want you to build up elements for a story (flash fiction) - which you will later use to make an outline. Use a maximum of 5 sentences unless instructed otherwise.'},
    ]

    
    for q in tqdm.tqdm(qs):
        messages.append({'role': 'user', 'content': q})
        print(messages[-1])
        output_message = llm.create_chat_completion(messages)['choices'][0]['message']
        print(output_message)
        messages.append(output_message)

    return messages, output_message['content']

Meta-Llama-3.1-8B-Instruct-Q8_0.gguf:   0%|          | 0.00/8.54G [00:00<?, ?B/s]

In [ ]:
messages, story = make_story('I\'ve had enough.')

print(story)

In [ ]:
def prompt_story_direct(idea):
    prompt = f"Write me a story around 700-1000 words based on the idea: '{idea}'"
    
    messages = [{'role':'user', 'content': prompt}]
    output_message = llm.create_chat_completion(messages)['choices'][0]['message']
    
    return output_message, output_message['content']

In [ ]:
_, story = prompt_story_direct('I\'ve had enough.')

print(story)